In [1]:
from anthropic import Anthropic
from typing import List, Dict
import base64
import json
import os
client = Anthropic()

In [2]:
# 로컬 파일을 base64로 인코딩
def get_base64_data(file_path):
    with open(file_path, "rb") as f:
        base64_data = base64.standard_b64encode(f.read()).decode("utf-8")
    return base64_data

In [180]:

system_prompt = """
당신은 박물관 도슨트입니다. 관람객의 질문에 친절하게 설명하세요.
채팅 창에 글씨가 너무 많으면 읽기 어려우니 가급적 5문장 이내로 답하세요.
"""

context_prompt = """
<context>
지금 제공된 국보/보물 이미지에 대해 설명을 진행해야 합니다.
</context>

<relic_information>
    <label>{label}</label>
    <content>{content}</content>
</relic_information>

<instructions>
if 사용자가 현재 작품에 대해 설명을 요청하면:
    <relic_information/>과 지금 제공된 국보/보물 이미지를 바탕으로 설명을 제공할 것
elif 사용자가 다음 작품을 보여달라고 요청하면:
    '[네] 다음 작품을 소개해드리겠습니다.'라고 답할 것([네]라는 문자열은 후속처리를 위해 사용되므로 변경하지 말것)
    
</instructions>
"""

In [181]:

class Relics:
    def __init__(self):
        self.load_relics()
        self.index = -1

    def load_relics(self):
        try:
            file_path = os.path.join("scrap", "combined_relics.json")
            with open(file_path, "r", encoding="utf-8") as f:
                self.relics: dict = json.load(f)
                for key, value in self.relics.items():
                    value["img"] = os.path.join(
                        "scrap", "relics", key, os.path.basename(value["img"])
                    )
                self.relic_ids = list(self.relics.keys())
        except Exception as e:
            import traceback

            msg = f"Error loading combined_relics.json: : {traceback.format_exc()}"
            print(msg)
            raise e

    def get_relic(self):
        self.relic_id = str(self.relic_ids[self.index])
        return (
            self.relics[self.relic_id]["label"],
            self.relics[self.relic_id]["content"],
            self.relics[self.relic_id]["img"],
        )

    def next_relic(self):
        self.index += 1
        return self.get_relic()


relics = Relics()
relics.get_relic()

({'명칭': '강세황 초상 자필본',
  '다른명칭': '姜世晃肖像 自筆本',
  '국적/시대': '한국 - 조선',
  '재질': '섬유 - 견',
  '작가': '강세황 姜世晃 (1713~1791)',
  '분류': '문화예술 - 서화 - 회화 - 일반회화',
  '크기': '세로(장황 포함) 150.0cm, 가로(장황 포함) 61.5cm, 세로(화면) 88.7cm, 가로(화면) 51.0cm',
  '지정문화유산': '보물',
  '소장품번호': '구 10562'},
 '',
 'scrap\\relics\\36560665\\koo010562-000-80000.jpg')

In [190]:
class DocentBot:

    hello = "안녕하세요! 도슨트봇입니다. 작품에 대해 궁금한 점이 있으시면 편하게 질문해주세요."

    def __init__(self, model_name: str = "claude-3-7-sonnet-20250219"):
        self.model = model_name
        self.messages = [{"role": "assistant", "content": self.hello}]
        self.relics = Relics()
        self.add_instruction()

    def get_image_path(self):
        return self.relics.get_relic()[2]
    
    def add_instruction(self):
        label, content, image_path = self.relics.next_relic()
        print(image_path)
        __context_prompt__ = context_prompt.format(label=label, content=content)
        self.messages.append({
            "role": "user", 
            "content": [
                {
                    "type": "image",
                    "source": {
                        "type": "base64",
                        "media_type": "image/jpeg",
                        "data": get_base64_data(image_path),
                    },
                },
                {
                    "type": "text",
                    "text": __context_prompt__,
                },
            ],
        })
        self.messages.append({"role": "assistant", "content": "<context>네, 지금 제공된 정보를 바탕으로 설명하겠습니다.</context>"})
    
    def create_response(self) -> str:
        try:
            # API 호출
            response = client.messages.create(
                max_tokens=1024,
                system=system_prompt,
                messages=self.messages,
                model=self.model,
                )
            # 응답 저장
            response_message = response.content[0].text
            return response_message
        except Exception as e:
            return f"Error: {str(e)}"
        
    def answer(self, user_input: str) -> str:
        self.messages.append({"role": "user", "content": user_input})
        response_message: str = self.create_response()
        if "[네]" in response_message:
            self.add_instruction()
            response_message = response_message.replace("[네]", "네")
        self.messages.append({"role": "assistant", "content": response_message})
        return response_message
    
    def get_conversation(self):
        conversation = []
        for message in self.messages:
            if isinstance(message['content'], list):
                text_message = message['content'][1]['text'].strip()
            else:
                text_message = message['content'].strip()
            if text_message.startswith("<context>"):
                continue
            conversation.append(text_message)
        return conversation

docent_bot = DocentBot()
docent_bot.get_image_path()

scrap\relics\348\bon001958-000-0001.jpg


'scrap\\relics\\348\\bon001958-000-0001.jpg'

In [191]:
x = docent_bot.get_conversation()

In [192]:
x

['안녕하세요! 도슨트봇입니다. 작품에 대해 궁금한 점이 있으시면 편하게 질문해주세요.']

In [193]:
response_message = docent_bot.answer("이 작품에 대해 설명해주세요.")

In [194]:
print(response_message)

이 작품은 국보로 지정된 '감산사 석조미륵보살입상'입니다. 통일신라 시대에 제작된 불상으로, 높이가 270cm에 달합니다.

신체와 광배(뒤의 후광)는 하나의 화강암으로 만들어졌고, 대좌는 별도로 제작하여 결합했습니다. 머리에는 높은 보관을 썼으며 중앙에 화불(작은 부처)이 표현되어 있습니다.

갸름한 얼굴에 미소를 띤 이 보살상은 목걸이, 팔찌 등 화려한 장식을 하고 있으며, 오른손은 자연스럽게 내려뜨리고 왼손은 들어 올려 손바닥을 보이고 있습니다.

광배 뒷면의 명문에 따르면 719년 김지성이 돌아가신 어머니를 위해 조성한 것으로, 사실적이고 관능적인 표현이 특징인 통일신라 8세기 불상의 대표적 사례입니다.


In [195]:
isinstance(docent_bot.messages[1]['content'], list)

docent_bot.messages[1]['content'][1]['text']

"\n<context>\n지금 제공된 국보/보물 이미지에 대해 설명을 진행해야 합니다.\n</context>\n\n<relic_information>\n    <label>{'명칭': '감산사 석조미륵보살입상', '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像', '전시명칭': '감산사미륵보살과 아미타불', '국적/시대': '한국 - 통일신라', '출토지': '출토지 - 경상북도', '재질': '돌 - 화강암', '분류': '종교신앙 - 불교 - 예배 - 불상', '크기': '높이 270.0cm', '지정문화유산': '국보', '소장품번호': '본관 1958', '전시위치': '불교조각'}</label>\n    <content>신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.</content>\n</relic_information>\n\n<instructions>\nif 사용자가 현재 작품에 대해 설명을 요청하면:\n    <relic_information/>과 지금 제공된 국보/보물 이미지를 바탕으로 설명을 제공할 것\ne

In [196]:
docent_bot.get_conversation()

['안녕하세요! 도슨트봇입니다. 작품에 대해 궁금한 점이 있으시면 편하게 질문해주세요.',
 '이 작품에 대해 설명해주세요.',
 "이 작품은 국보로 지정된 '감산사 석조미륵보살입상'입니다. 통일신라 시대에 제작된 불상으로, 높이가 270cm에 달합니다.\n\n신체와 광배(뒤의 후광)는 하나의 화강암으로 만들어졌고, 대좌는 별도로 제작하여 결합했습니다. 머리에는 높은 보관을 썼으며 중앙에 화불(작은 부처)이 표현되어 있습니다.\n\n갸름한 얼굴에 미소를 띤 이 보살상은 목걸이, 팔찌 등 화려한 장식을 하고 있으며, 오른손은 자연스럽게 내려뜨리고 왼손은 들어 올려 손바닥을 보이고 있습니다.\n\n광배 뒷면의 명문에 따르면 719년 김지성이 돌아가신 어머니를 위해 조성한 것으로, 사실적이고 관능적인 표현이 특징인 통일신라 8세기 불상의 대표적 사례입니다."]

In [197]:
response_message = docent_bot.answer("기분이 묘해요.")

In [198]:
print(response_message)

작품을 보시고 기분이 묘하시다니, 충분히 이해됩니다. 이 미륵보살상은 1300년 전 통일신라 시대 작품으로, 시간을 초월한 평온한 미소와 우아한 자태가 현대인에게도 특별한 감정을 불러일으킵니다.

고대 예술품이 주는 신비로움, 장엄함, 그리고 영적인 느낌이 복합적으로 다가올 수 있어요. 특히 어머니를 위해 만들어진 불상이라는 배경 이야기가 더해져 감정적인 울림을 주기도 합니다.

무엇이 특히 인상적으로 다가오셨나요?


In [199]:
response_message = docent_bot.answer("도슨트 님은 어느 부분이 제일 매력적이에요?")

In [200]:
response_message

'제가 이 감산사 석조미륵보살입상에서 가장 매력적이라고 느끼는 부분은 그 섬세한 표정과 우아한 신체 표현입니다. \n\n특히 얼굴에 머무는 미소가 1300년이 지난 지금까지도 생생하게 전해져 오는 점이 경이롭습니다. 살짝 올라간 입꼬리와 온화한 눈매에서 보살의 자비로움이 느껴지죠.\n\n또한 얇은 법의(옷)를 통해 드러나는 신체의 유려한 곡선미는 통일신라 불상 조각의 뛰어난 예술성을 보여줍니다. 장식과 의복의 흐름이 자연스러우면서도 품격 있게 표현된 점이 정말 매력적입니다.'

In [201]:
response_message = docent_bot.answer("다음 작품 소개해주세요.")

scrap\relics\349\bon001959-00-01.jpg


In [202]:
response_message

'네 다음 작품을 소개해드리겠습니다.'

In [203]:
response_message = docent_bot.answer("이 작품은 뭐에요?")

In [204]:
response_message

"이 작품은 '감산사 석조아미타불입상'으로, 국보로 지정된 통일신라 시대의 불상입니다. \n\n1915년 경주 감산사터에서 앞서 보신 미륵보살상과 함께 발견되었습니다. 전체 높이는 275cm로, 광배와 대좌가 하나의 화강암으로 만들어졌습니다.\n\n광배에는 719년 김지성이 돌아가신 부모님을 위해 감산사를 짓고 미륵보살과 아미타여래를 조성했다는 명문이 새겨있어, 통일신라 불상 연구의 중요한 기준작으로 여겨집니다.\n\n넓은 얼굴에 근엄한 표정을 짓고 있으며, 오른손은 들어 손바닥을 앞으로 향하고, 왼손은 아래로 늘어뜨렸습니다. 얇은 법의를 통해 신체의 곡선이 잘 드러나는 8세기 신라 불상의 전형적인 작품입니다."

In [205]:
response_message = docent_bot.answer("아까 작품이랑 비교하면 어떤 느낌이 차이가 나요?")

In [206]:
response_message

'두 작품을 비교하면 확실히 다른 느낌이 있습니다.\n\n앞서 본 미륵보살상은 보살로서 화려한 보관과 장신구로 장식되어 있고, 얼굴이 갸름하며 부드러운 미소를 띠고 있어 친근하고 우아한 느낌을 줍니다.\n\n반면 이 아미타불입상은 부처님으로서 장신구 없이 소박하고, 넓적한 얼굴에 보다 엄숙하고 근엄한 표정을 지니고 있습니다. 머리는 나발(작은 곱슬머리)로 표현되어 있고 육계가 크고 편평합니다.\n\n두 작품 모두 같은 시기(719년)에 같은 인물(김지성)이 부모를 위해 조성했지만, 하나는 자비로운 보살의 이미지로, 다른 하나는 깨달음을 완성한 부처님의 이미지로 의도적으로 다르게 표현되었습니다.'

In [207]:
docent_bot.messages

[{'role': 'assistant',
  'content': '안녕하세요! 도슨트봇입니다. 작품에 대해 궁금한 점이 있으시면 편하게 질문해주세요.'},
 {'role': 'user',
  'content': [{'type': 'image',
    'source': {'type': 'base64',
     'media_type': 'image/jpeg',
     'data': '...'}},
   {'type': 'text',
    'text': "\n<context>\n지금 제공된 국보/보물 이미지에 대해 설명을 진행해야 합니다.\n</context>\n\n<relic_information>\n    <label>{'명칭': '감산사 석조미륵보살입상', '다른명칭': '국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像', '전시명칭': '감산사미륵보살과 아미타불', '국적/시대': '한국 - 통일신라', '출토지': '출토지 - 경상북도', '재질': '돌 - 화강암', '분류': '종교신앙 - 불교 - 예배 - 불상', '크기': '높이 270.0cm', '지정문화유산': '국보', '소장품번호': '본관 1958', '전시위치': '불교조각'}</label>\n    <content>신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷

In [208]:
#docent_bot.messages[1]

In [149]:
docent_bot.messages[1]['content'][0]['source']['data'] == docent_bot.messages[-7]['content'][0]['source']['data']

False

In [153]:
docent_bot.get_conversation()

TypeError: string indices must be integers, not 'str'

In [2]:
label = {
    "명칭": "감산사 석조미륵보살입상",
    "다른명칭": "국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像",
    "전시명칭": "감산사미륵보살과 아미타불",
    "국적/시대": "한국 - 통일신라",
    "출토지": "출토지 - 경상북도",
    "재질": "돌 - 화강암",
    "분류": "종교신앙 - 불교 - 예배 - 불상",
    "크기": "높이 270.0cm",
    "지정문화유산": "국보",
    "소장품번호": "본관 1958",
    "전시위치": "불교조각"
},
content= "신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.",
        

In [97]:
from anthropic import Anthropic
from typing import List, Dict
import base64
import json
import os
import tools

client = Anthropic()

context_prompt2 = """
<context>
[시스템 운영자]지금 제공된 국보/보물 이미지에 대해 설명을 진행해야 합니다.
</context>

<relic_information>
    <label>{label}</label>
    <content>{content}</content>
</relic_information>

<instructions>
다음 단계로 사고할 것:
1. <user/>의 메시지를 다음 case 중 하나로 분류할 것:
    case1: 작품 검색 요청
    case2: 그 밖의 내용
2. case1에 해당하면 ''를 출력할 것 
3. case2에 해당하면 <relic_information/>과 지금 제공된 국보/보물 이미지를 바탕으로 설명할 것
</instructions>

<user>
{user_message}
</user>

<response_format>
최종 출력결과는 <json>태그로 감싸서 다음 JSON 포맷으로 출력할 것
{{
    "다음 중 하나를 고를 것": <case1|case2>,
    "대화내용":,
}}    
</response_format>
"""

context_prompt2 = context_prompt2.format(label=label, content=content, user_message="조선 시대 회회보고 싶어")
context_prompt2



'\n<context>\n[시스템 운영자]지금 제공된 국보/보물 이미지에 대해 설명을 진행해야 합니다.\n</context>\n\n<relic_information>\n    <label>({\'명칭\': \'감산사 석조미륵보살입상\', \'다른명칭\': \'국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像\', \'전시명칭\': \'감산사미륵보살과 아미타불\', \'국적/시대\': \'한국 - 통일신라\', \'출토지\': \'출토지 - 경상북도\', \'재질\': \'돌 - 화강암\', \'분류\': \'종교신앙 - 불교 - 예배 - 불상\', \'크기\': \'높이 270.0cm\', \'지정문화유산\': \'국보\', \'소장품번호\': \'본관 1958\', \'전시위치\': \'불교조각\'},)</label>\n    <content>(\'신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.\',)</content>\n</relic_information>\n\n<instructions>\n다음 단계로 사고할 것:\n1. <user/>의 

In [98]:
prompt = [
    {"role": "user","content": context_prompt2},        
    {"role": "assistant","content": "<json>"}
]

In [99]:
prompt

[{'role': 'user',
  'content': '\n<context>\n[시스템 운영자]지금 제공된 국보/보물 이미지에 대해 설명을 진행해야 합니다.\n</context>\n\n<relic_information>\n    <label>({\'명칭\': \'감산사 석조미륵보살입상\', \'다른명칭\': \'국보 경주 감산사 석조 미륵보살 입상(1962), 慶州 甘山寺 石造彌勒菩薩立像\', \'전시명칭\': \'감산사미륵보살과 아미타불\', \'국적/시대\': \'한국 - 통일신라\', \'출토지\': \'출토지 - 경상북도\', \'재질\': \'돌 - 화강암\', \'분류\': \'종교신앙 - 불교 - 예배 - 불상\', \'크기\': \'높이 270.0cm\', \'지정문화유산\': \'국보\', \'소장품번호\': \'본관 1958\', \'전시위치\': \'불교조각\'},)</label>\n    <content>(\'신체와 광배는 하나의 돌로 제작하고, 별도로 제작한 대좌에 결합시켰다. 이러한 형식은 감산사 절터에서 함께 수습된 <아미타불>과 같다. 머리에는 높은 보관을 썼는데 중앙에 화불(化佛)이 있다. 얼굴은 갸름하나 살이 올라 있고 눈과 입에 미소가 어려 있다. 목에는 삼도가 뚜렷하며 목걸이, 팔찌, 영락 장식 등으로 신체를 화려하게 장식하고 있다. 오른손은 자연스럽게 내려뜨리고 있고, 왼손은 들어 올려 손바닥을 보이고 있다. 팔목에는 천의가 걸쳐져 있는데, 법의(法衣)는 얇아서 신체의 풍만하고 유려한 곡선을 더욱 살려주고 있다. 광배는 배모양에 신체를 모두 감싸는 주형거신광(舟形擧身光)으로, 세 가닥의 선으로 두광과 신광을 구분하였다. 광배 뒷면에는 명문이 새겨져 있는데, 이를 통해 719년 김지성(金志誠)이 돌아가신 어머니를 위해 조성한 미륵보살상임을 알 수 있다. 표현이 사실적이고 관능적인 모습을 한 통일신라 8세기 불상의 대표적인 사례이다.\',)</content>\n</relic_information>\n\n<instruction

In [100]:
system_prompt = """
- 당신은 e-박물관 도슨트입니다. 사용자의 질문에 친절하게 설명하세요.
- 사용자는 채팅 창에서 왼쪽의 박물관 이미지를 감상 중입니다. 이미지 아래의 [이전]과 [다음]버튼으로 내비케이션 할 수 있습니다.
- 채팅 창에 글씨가 너무 많으면 읽기 어려우니 가급적 3문장 이내로 답하세요.
- 현장에서 설명하는 것처럼 말해야 하므로 번호, 대시, 불릿 포인트 등을 사용하지 마세요.
"""


response = client.messages.create(
    max_tokens=1024,
    system=system_prompt,
    messages=prompt,
    model="claude-3-7-sonnet-20250219",
    stop_sequences=["</json>"],
)
# 응답 저장
response_message = response.content[0].text

In [101]:
from pprint import pprint
pprint(json.loads(response_message))

{'다음 중 하나를 고를 것': 'case1', '대화내용': ''}
